# Análise de Clusters em Indicadores Socioeconômicos e Suicídios por País (1990–2019)

## É importante ressaltar:

#### 1. A principal estratificação do dataset (AgeGroup) se perde ao analisarmos a contagem total de suicídios por ano em determinado país

#### 2. Gross Domestic Product (GDP) Per Capita é medido em dólar e o valor do dólar muda ao longo dos anos

#### 3. Os clusters apresentarão variações nos indicadores socioeconômicos e na contagem de suicídio, uma vez que países podem mudar de cluster em diferentes décadas. Existirão também outros fatores não listados que afetam as métricas dos grupos.

#### 4. A convenção utilizada para avaliar a inflação foi:
* Abaixo de 4,5% = baixa;
* Entre 4,5% e 10% = média;
* Entre 10% e 50% = alta;
* Acima de 50% = hiperinflação.

#### 5. Para a análise de clusters, utilizaremos conceitos como centro e periferia na economia global, visto que essa dinâmica proporciona a análise de relações entre os países e remove de cena termos como países "emergentes", "subdesenvolvidos" e "em desenvolvimento".

In [1]:
# importando bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px


In [2]:
# carregando o dataset
df = pd.read_csv('suicide_rates_limpo.csv')

In [3]:
# seleção de colunas relevantes
cols = ['CountryName', 'Year', 'GDPPerCapita', 'InflationRate', 'EmploymentPopulationRatio', 'SuicideCount']
df_filtered = df[cols]

# remoção de valores nulos
df_filtered = df_filtered.dropna()

# agrupar por país e ano, somando SuicideCount e extraindo a média dos indicadores econômicos
df_aggregated = df_filtered.groupby(['CountryName', 'Year']).agg({
    'SuicideCount': 'sum',
    'GDPPerCapita': 'mean',
    'InflationRate': 'mean',
    'EmploymentPopulationRatio': 'mean'
}).reset_index()


In [4]:
# coluna de década
df_aggregated['Decade'] = pd.cut(df_aggregated['Year'],
                                  bins=[1989, 1999, 2009, 2019, 2029],
                                  labels=['1990s', '2000s', '2010s', '2020s'],
                                  right=True)

# removendo anos 2020, 2021 e 2022
df_aggregated = df_aggregated[df_aggregated['Decade'] != '2020s']

# 3 datasets, um para cada década
df_90s = df_aggregated[df_aggregated['Decade'] == '1990s'].copy()
df_00s = df_aggregated[df_aggregated['Decade'] == '2000s'].copy()
df_10s = df_aggregated[df_aggregated['Decade'] == '2010s'].copy()

# remoção de coluna Decade para limpar os datasets
for df_decade in [df_90s, df_00s, df_10s]:
    df_decade.drop(columns='Decade', inplace=True)



In [5]:
def process_clusters(df_decade, n_clusters=4):
    features = ['GDPPerCapita', 'InflationRate', 'EmploymentPopulationRatio', 'SuicideCount']
    
    df_grouped = df_decade.groupby('CountryName')[features].mean().reset_index() # média dos indicadores de cada país ao longo da década correspondente 
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_grouped[features])
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    clusters = kmeans.fit_predict(X_scaled)
    
    df_grouped['Cluster'] = clusters
    
    return df_grouped


def plot_clusters_on_map(df_grouped, decade_label, cluster_labels=None):
    if cluster_labels:
        df_grouped['ClusterLabel'] = df_grouped['Cluster'].map(cluster_labels)
        color_col = 'ClusterLabel'
        color_seq = px.colors.qualitative.Bold
    else:
        color_col = 'Cluster'
        color_seq = None

    # Plotagem
    fig = px.choropleth(
        df_grouped,
        locations='CountryName',
        locationmode='country names',
        color=color_col,
        title=f'Clusters por País – {decade_label}',
        color_discrete_sequence=color_seq
    )
    fig.show()



In [6]:
# resumo de clusters por década
def summarize_clusters(df_clustered, decade_label):
    summary = df_clustered.groupby('Cluster')[
        ['GDPPerCapita', 'InflationRate', 'EmploymentPopulationRatio', 'SuicideCount']
    ].mean().round(2)
    
    print(f"\nDécada: {decade_label}")
    display(summary)

In [7]:
df_90s_clustered = process_clusters(df_90s)
summarize_clusters(df_90s_clustered, '1990s')


Década: 1990s


,GDPPerCapita,InflationRate,EmploymentPopulationRatio,SuicideCount
Cluster,,,,
0,4968.32,19.30,50.02,1015.40
1,26350.85,2.58,57.40,4437.21
2,1684.33,685.44,54.93,5996.80
3,5358.50,64.96,63.88,3924.29


Para a década de 1990, é possível perceber:

* Cluster 0: Menor índice médio de suicídios, alta inflação, 2º menor GDP Per Capita;
* Cluster 1: Segundo maior índice médio de suicídios, baixa inflação, maior GDP Per Capita;
* Cluster 2: Maior índice médio de suicídios, hiperinflação, menor GDP Per Capita;
* Cluster 3: Segundo menor índice médio de suicídios, hiperinflação, 2º maior GDP Per Capita.

A partir disso, podemos supor:

* Cluster 0 - Periferia na economia global, baixo suicídio;
* Cluster 1 - Centro na economia global, médio suicídio;
* Cluster 2 - Periferia na economia global, alto suicídio;
* Cluster 3 - Periferia na economia global, médio suicídio.

In [8]:
cluster_labels_90s = {
    0: 'Periferia, baixo suicídio',
    1: 'Centro, médio suicídio',
    2: 'Periferia, alto suicídio',
    3: 'Periferia, médio suicídio'
}

df_90s_clustered['ClusterLabel'] = df_90s_clustered['Cluster'].map(cluster_labels_90s)

plot_clusters_on_map(df_90s_clustered, '1990s', cluster_labels_90s)


In [9]:
df_00s_clustered = process_clusters(df_00s)
summarize_clusters(df_00s_clustered, '2000s')


Década: 2000s


,GDPPerCapita,InflationRate,EmploymentPopulationRatio,SuicideCount
Cluster,,,,
0,6831.05,10.52,46.31,1511.05
1,28421.27,5.43,58.56,37005.30
2,7528.18,5.16,58.02,1242.43
3,39471.67,2.39,61.39,1801.61


Para a década de 2000, é possível perceber:

* Cluster 0: 2º menor índice médio de suicídios, alta inflação, menor GDP Per Capita;
* Cluster 1: Maior índice médio de suicídios, média inflação, 2º maior GDP Per Capita;
* Cluster 2: Menor índice médio de suicídios, média inflação, 2º menor GDP Per Capita;
* Cluster 3: 2º maior índice médio de suicídios, baixa inflação, maior GDP Per Capita;

Observação: dois clusters (1 e 3) apresentam GDP Per Capita acima de 25 mil dólares. É um grande indicativo que ambos sejam compostos por países no centro da economia global, apesar de certa discrepância entre os demais indicativos socioeconômicos e índice médio de suicídio.

A partir disso, podemos supor:

* Cluster 0 - Periferia na economia global, médio suicídio;
* Cluster 1 - Centro na economia global, alto suicídio;
* Cluster 2 - Periferia na economia global, baixo suicídio;
* Cluster 3 - Centro na economia global, médio suicídio.

In [10]:
cluster_labels_00s = {
    0: 'Periferia, médio suicídio',
    1: 'Centro, alto suicídio',
    2: 'Periferia, baixo suicídio',
    3: 'Centro, médio suicídio'
}

df_00s_clustered['ClusterLabel'] = df_00s_clustered['Cluster'].map(cluster_labels_00s)

plot_clusters_on_map(df_00s_clustered, '2000s', cluster_labels_00s)


In [11]:
df_10s_clustered = process_clusters(df_10s)
summarize_clusters(df_10s_clustered, '2010s')


Década: 2010s


,GDPPerCapita,InflationRate,EmploymentPopulationRatio,SuicideCount
Cluster,,,,
0,53690.60,1.58,63.52,1328.08
1,11936.07,3.20,53.95,1384.25
2,6459.24,17.28,49.80,2510.55
3,36634.03,3.03,58.56,30939.23


Para a década de 2010, é possível perceber:

* Cluster 0: Menor índice médio de suicídios, baixa inflação, maior GDP Per Capita;
* Cluster 1: 2º menor índice médio de suicídios, inflação sob controle, 2º menor GDP Per Capita;
* Cluster 2: 2º maior índice médio de suicídios, alta inflação, menor GDP Per Capita;
* Cluster 3: Maior índice médio de suicídios, inflação sob controle, 2º maior GDP Per Capita.

Observação: dois clusters apresentam o GDP Per Capita acima de 35 mil dólares. É um grande indicativo que ambos sejam compostos por países no centro da economia global, apesar de certa discrepância entre os demais indicativos socioeconômicos e índice médio de suicídio.

A partir disso, podemos supor:

* Cluster 0 - Centro na economia global, baixo suicídio;
* Cluster 1 - Periferia na economia global, baixo suicídio;
* Cluster 2 - Periferia na economia global, médio suicídio;
* Cluster 3 - Centro na economia global, alto suicídio.

In [12]:
cluster_labels_10s = {
    0: 'Centro, baixo suicídio',
    1: 'Periferia, baixo suicídio',
    2: 'Periferia, médio suicídio',
    3: 'Centro, alto suicídio'
}

df_10s_clustered['ClusterLabel'] = df_10s_clustered['Cluster'].map(cluster_labels_10s)

plot_clusters_on_map(df_10s_clustered, '2010s', cluster_labels_10s)
